In [2]:
import oic_dwic
from script import utils

In [3]:
DWIC = oic_dwic.InterAtomicContact(
    pathfiles="../test/",
    filename="DWIC.csv",
    ligand_format="mol2",
    amino_acid_classes=utils.amino_acid_classes_DWIC,
    cutoff=12,
    feature_type="DWIC",
    exp=2,
)

OIC = oic_dwic.InterAtomicContact(
    pathfiles="../test/",
    filename="OIC.csv",
    ligand_format="mol2",
    amino_acid_classes=utils.amino_acid_classes_OIC,
    cutoff=12,
    feature_type="OIC",
    exp=None,
)

In [4]:
DWIC.generate_features(n_jobs=4)
OIC.generate_features(n_jobs=4)

Time: 0.0:0.0:7.81
Time: 0.0:0.0:2.30
